In [2]:
!pip install -q datasets transformers onnx onnxruntime huggingface_hub

     |████████████████████████████████| 346 kB 18.9 MB/s 
     |████████████████████████████████| 4.2 MB 59.0 MB/s 
     |████████████████████████████████| 12.8 MB 58.0 MB/s 
     |████████████████████████████████| 5.2 MB 65.1 MB/s 
     |████████████████████████████████| 86 kB 6.1 MB/s 
     |████████████████████████████████| 212 kB 70.2 MB/s 
     |████████████████████████████████| 140 kB 59.7 MB/s 
     |████████████████████████████████| 1.1 MB 56.2 MB/s 
     |████████████████████████████████| 86 kB 7.9 MB/s 
     |████████████████████████████████| 596 kB 68.2 MB/s 
     |████████████████████████████████| 127 kB 76.7 MB/s 
     |████████████████████████████████| 6.6 MB 57.7 MB/s 
     |████████████████████████████████| 271 kB 78.7 MB/s 
     |████████████████████████████████| 94 kB 4.1 MB/s 
     |████████████████████████████████| 144 kB 65.7 MB/s 
     |████████████████████████████████| 112 kB 76.8 MB/s 
ERROR: pip's dependency resolver does not currently take into account all the

In [20]:
import numpy as np

from transformers import AutoTokenizer, AutoModelForSequenceClassification, TrainingArguments, Trainer
from datasets import load_dataset, load_metric

import torch

device = "cuda:0" if torch.cuda.is_available() else "cpu"
print(device)

cuda:0


In [6]:
dataset = load_dataset("ziq/depression_tweet")

Using custom data configuration ziq--depression_tweet-09dd1a73b951cbf2


Extracting data files:   0%|          | 0/3 [00:00<?, ?it/s]

Dataset csv downloaded and prepared to /root/.cache/huggingface/datasets/ziq___csv/ziq--depression_tweet-09dd1a73b951cbf2/0.0.0/433e0ccc46f9880962cc2b12065189766fbb2bee57a221866138fb9203c83519. Subsequent calls will reuse this data.


  0%|          | 0/3 [00:00<?, ?it/s]

In [9]:
dataset["train"][100]

{'label': 0,
 'text': 'august th prediction arg tiktok live people saying today government going announce fake alien invasion new world order thoughts'}

In [70]:
model_name = 'microsoft/xtremedistil-l6-h384-uncased'

tokenizer = AutoTokenizer.from_pretrained(model_name)

def tokenize_function(examples):
    return tokenizer(examples["text"], padding="max_length", truncation=True, max_length=64)

cols = dataset["train"].column_names
cols.remove("label")
dataset_ = dataset.map(tokenize_function, batched=True, remove_columns=cols)
dataset_

Could not locate the tokenizer configuration file, will try to use the model config instead.
loading configuration file https://huggingface.co/microsoft/xtremedistil-l6-h384-uncased/resolve/main/config.json from cache at /root/.cache/huggingface/transformers/31d6577412393ebb07c02de876b2d1397fcae2d85cb053b588145f6869ab1a15.44cd178af39e607af310bc4cc48a944f5e5f746b372c161b32511f0fd585789b
Model config BertConfig {
  "_name_or_path": "microsoft/xtremedistil-l6-h384-uncased",
  "architectures": [
    "BertModel"
  ],
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "gradient_checkpointing": false,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 384,
  "initializer_range": 0.02,
  "intermediate_size": 1536,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_heads": 12,
  "num_hidden_layers": 6,
  "pad_token_id": 0,
  "position_embedding_type": "absolute",
  "transformers_version": "4.19.2",
  "typ

  0%|          | 0/28 [00:00<?, ?ba/s]

  0%|          | 0/10 [00:00<?, ?ba/s]

  0%|          | 0/10 [00:00<?, ?ba/s]

DatasetDict({
    train: Dataset({
        features: ['label', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 27613
    })
    test: Dataset({
        features: ['label', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 9205
    })
    validation: Dataset({
        features: ['label', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 9204
    })
})

In [71]:
dataset['train'], dataset_["train"]

(Dataset({
     features: ['text', 'label'],
     num_rows: 27613
 }), Dataset({
     features: ['label', 'input_ids', 'token_type_ids', 'attention_mask'],
     num_rows: 27613
 }))

In [72]:
dataset_ = dataset_.rename_column("label", "labels")

In [77]:
model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=2)

loading configuration file https://huggingface.co/microsoft/xtremedistil-l6-h384-uncased/resolve/main/config.json from cache at /root/.cache/huggingface/transformers/31d6577412393ebb07c02de876b2d1397fcae2d85cb053b588145f6869ab1a15.44cd178af39e607af310bc4cc48a944f5e5f746b372c161b32511f0fd585789b
Model config BertConfig {
  "_name_or_path": "microsoft/xtremedistil-l6-h384-uncased",
  "architectures": [
    "BertModel"
  ],
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "gradient_checkpointing": false,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 384,
  "initializer_range": 0.02,
  "intermediate_size": 1536,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_heads": 12,
  "num_hidden_layers": 6,
  "pad_token_id": 0,
  "position_embedding_type": "absolute",
  "transformers_version": "4.19.2",
  "type_vocab_size": 2,
  "use_cache": true,
  "vocab_size": 30522
}

loading weights file https://

In [73]:
small_train_dataset = dataset_["train"].shuffle(seed=42).select(range(1000))
small_eval_dataset = dataset_["test"].shuffle(seed=42).select(range(1000))

In [78]:
training_args = TrainingArguments(
    output_dir="test_trainer", 
    evaluation_strategy="epoch",
    per_device_train_batch_size=128, 
    num_train_epochs=4,
    learning_rate=3e-05
)

PyTorch: setting up devices
The default value for the training argument `--report_to` will change in v5 (from all installed integrations to none). In v5, you will need to use `--report_to all` to get the same behavior as now. You should start updating your code and make this info disappear :-).


In [89]:
np.where(5 > 0.5, 1, 0).item()

1

In [112]:
metric = load_metric("accuracy")

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    return metric.compute(predictions=predictions, references=labels)

In [113]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=small_train_dataset,
    eval_dataset=small_eval_dataset,
    compute_metrics=compute_metrics,
)

In [114]:
trainer.train()

/usr/local/lib/python3.7/dist-packages/transformers/optimization.py:309: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  FutureWarning,
***** Running training *****
  Num examples = 1000
  Num Epochs = 4
  Instantaneous batch size per device = 128
  Total train batch size (w. parallel, distributed & accumulation) = 128
  Gradient Accumulation steps = 1
  Total optimization steps = 32


Epoch,Training Loss,Validation Loss,Accuracy
1,No log,0.336539,0.884000
2,No log,0.336130,0.888000
3,No log,0.328727,0.886000
4,No log,0.331152,0.885000


***** Running Evaluation *****
  Num examples = 1000
  Batch size = 8
***** Running Evaluation *****
  Num examples = 1000
  Batch size = 8
***** Running Evaluation *****
  Num examples = 1000
  Batch size = 8
***** Running Evaluation *****
  Num examples = 1000
  Batch size = 8


Training completed. Do not forget to share your model on huggingface.co/models =)




TrainOutput(global_step=32, training_loss=0.17644299566745758, metrics={'train_runtime': 11.2211, 'train_samples_per_second': 356.472, 'train_steps_per_second': 2.852, 'total_flos': 16582904832000.0, 'train_loss': 0.17644299566745758, 'epoch': 4.0})

In [122]:
from huggingface_hub import notebook_login

notebook_login()

Login successful
Your token has been saved to /root/.huggingface/token
Authenticated through git-credential store but this isn't the helper defined on your machine.
You might have to re-authenticate when pushing to the Hugging Face Hub. Run the following command in your terminal in case you want to set this credential helper as the default

git config --global credential.helper store


In [126]:
training_args = TrainingArguments("depression_tweet",
                                  per_device_train_batch_size=128, 
                                  num_train_epochs=4,
                                  learning_rate=3e-05,
                                  evaluation_strategy="epoch",
                                  push_to_hub=True
                                  )

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=dataset_['train'],
    eval_dataset=dataset_['validation'],
    compute_metrics=compute_metrics
)

PyTorch: setting up devices
The default value for the training argument `--report_to` will change in v5 (from all installed integrations to none). In v5, you will need to use `--report_to all` to get the same behavior as now. You should start updating your code and make this info disappear :-).
Cloning https://huggingface.co/ziq/depression_tweet into local empty directory.


In [127]:
trainer.train()

/usr/local/lib/python3.7/dist-packages/transformers/optimization.py:309: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  FutureWarning,
***** Running training *****
  Num examples = 27613
  Num Epochs = 4
  Instantaneous batch size per device = 128
  Total train batch size (w. parallel, distributed & accumulation) = 128
  Gradient Accumulation steps = 1
  Total optimization steps = 864


Epoch,Training Loss,Validation Loss,Accuracy
1,No log,0.136879,0.949696
2,No log,0.158841,0.955237
3,0.051400,0.164716,0.956215
4,0.051400,0.160558,0.956541


***** Running Evaluation *****
  Num examples = 9204
  Batch size = 8
***** Running Evaluation *****
  Num examples = 9204
  Batch size = 8
Saving model checkpoint to depression_tweet/checkpoint-500
Configuration saved in depression_tweet/checkpoint-500/config.json
Model weights saved in depression_tweet/checkpoint-500/pytorch_model.bin
***** Running Evaluation *****
  Num examples = 9204
  Batch size = 8
***** Running Evaluation *****
  Num examples = 9204
  Batch size = 8


Training completed. Do not forget to share your model on huggingface.co/models =)




TrainOutput(global_step=864, training_loss=0.047407187797405104, metrics={'train_runtime': 238.3502, 'train_samples_per_second': 463.402, 'train_steps_per_second': 3.625, 'total_flos': 457903751126016.0, 'train_loss': 0.047407187797405104, 'epoch': 4.0})

In [128]:
trainer.push_to_hub()

Saving model checkpoint to depression_tweet
Configuration saved in depression_tweet/config.json
Model weights saved in depression_tweet/pytorch_model.bin


Upload file pytorch_model.bin:   0%|          | 3.34k/86.7M [00:00<?, ?B/s]

Upload file runs/Jun06_09-03-20_da049709c8f8/events.out.tfevents.1654506216.da049709c8f8.77.36:  63%|######3  …

remote: Enforcing permissions...        
remote: Allowed refs: all        
To https://huggingface.co/ziq/depression_tweet
   1e4f6a7..7f2b66a  main -> main

Dropping the following result as it does not have all the necessary fields:
{'task': {'name': 'Text Classification', 'type': 'text-classification'}, 'metrics': [{'name': 'Accuracy', 'type': 'accuracy', 'value': 0.9565406345067362}]}
remote: Enforcing permissions...        
remote: Allowed refs: all        
To https://huggingface.co/ziq/depression_tweet
   7f2b66a..242c152  main -> main



'https://huggingface.co/ziq/depression_tweet/commit/7f2b66af0aff438a73baac0bd613c2fbf64686ad'

Export PyTorch model to ONNX format for serving with ONNX Runtime Web 

In [129]:
import transformers
import transformers.convert_graph_to_onnx as onnx_convert
from pathlib import Path

In [130]:
from transformers import AutoModelForSequenceClassification, AutoTokenizer
# tokenizer = AutoTokenizer.from_pretrained("bergum/xtremedistil-l6-h384-go-emotion")
model = AutoModelForSequenceClassification.from_pretrained("ziq/depression_tweet")

https://huggingface.co/ziq/depression_tweet/resolve/main/config.json not found in cache or force_download set to True, downloading to /root/.cache/huggingface/transformers/tmpx4s3wt3e


Downloading:   0%|          | 0.00/747 [00:00<?, ?B/s]

storing https://huggingface.co/ziq/depression_tweet/resolve/main/config.json in cache at /root/.cache/huggingface/transformers/94853bf532e251fb6ac5f311de76fd5597c85d2eb46b385901d5685b9b198406.ee4e64b6e9c036dd3998e5e81641c0edd36292b8a93ec3d38d7675a24bfab952
creating metadata file for /root/.cache/huggingface/transformers/94853bf532e251fb6ac5f311de76fd5597c85d2eb46b385901d5685b9b198406.ee4e64b6e9c036dd3998e5e81641c0edd36292b8a93ec3d38d7675a24bfab952
loading configuration file https://huggingface.co/ziq/depression_tweet/resolve/main/config.json from cache at /root/.cache/huggingface/transformers/94853bf532e251fb6ac5f311de76fd5597c85d2eb46b385901d5685b9b198406.ee4e64b6e9c036dd3998e5e81641c0edd36292b8a93ec3d38d7675a24bfab952
Model config BertConfig {
  "_name_or_path": "ziq/depression_tweet",
  "architectures": [
    "BertForSequenceClassification"
  ],
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "gradient_checkpointing": false,
  "hidden_act": "gelu",
  "hidden_d

Downloading:   0%|          | 0.00/86.7M [00:00<?, ?B/s]

storing https://huggingface.co/ziq/depression_tweet/resolve/main/pytorch_model.bin in cache at /root/.cache/huggingface/transformers/49300672ec8b473410b611196bbd5fb1ade78762268e02e834005d413b07ff24.1eabf4fe0ededc89bfd7f517e7607fe26761ebd36b79f3e39fe2fab1b430f290
creating metadata file for /root/.cache/huggingface/transformers/49300672ec8b473410b611196bbd5fb1ade78762268e02e834005d413b07ff24.1eabf4fe0ededc89bfd7f517e7607fe26761ebd36b79f3e39fe2fab1b430f290
loading weights file https://huggingface.co/ziq/depression_tweet/resolve/main/pytorch_model.bin from cache at /root/.cache/huggingface/transformers/49300672ec8b473410b611196bbd5fb1ade78762268e02e834005d413b07ff24.1eabf4fe0ededc89bfd7f517e7607fe26761ebd36b79f3e39fe2fab1b430f290
All model checkpoint weights were used when initializing BertForSequenceClassification.

All the weights of BertForSequenceClassification were initialized from the model checkpoint at ziq/depression_tweet.
If your task is similar to the task the model of the check

In [131]:
pipeline = transformers.pipeline("text-classification",model=model,tokenizer=tokenizer)

In [134]:
pipeline("I am depressed")

[{'label': 'LABEL_1', 'score': 0.7148955464363098}]

In [135]:
onnx_convert.convert_pytorch(pipeline, opset=11, output=Path("depression.onnx"), use_external_format=False)

Using framework PyTorch: 1.11.0+cu113
Found input input_ids with shape: {0: 'batch', 1: 'sequence'}
Found input token_type_ids with shape: {0: 'batch', 1: 'sequence'}
Found input attention_mask with shape: {0: 'batch', 1: 'sequence'}
Found output output_0 with shape: {0: 'batch'}
Ensuring inputs are in correct order
position_ids is not present in the generated input list.
Generated inputs order: ['input_ids', 'attention_mask', 'token_type_ids']


In [136]:
from onnxruntime.quantization import quantize_dynamic, QuantType
quantize_dynamic("depression.onnx", "depression-int8.onnx", 
                 weight_type=QuantType.QUInt8)

Ignore MatMul due to non constant B: /[MatMul_68]
Ignore MatMul due to non constant B: /[MatMul_73]
Ignore MatMul due to non constant B: /[MatMul_162]
Ignore MatMul due to non constant B: /[MatMul_167]
Ignore MatMul due to non constant B: /[MatMul_256]
Ignore MatMul due to non constant B: /[MatMul_261]
Ignore MatMul due to non constant B: /[MatMul_350]
Ignore MatMul due to non constant B: /[MatMul_355]
Ignore MatMul due to non constant B: /[MatMul_444]
Ignore MatMul due to non constant B: /[MatMul_449]
Ignore MatMul due to non constant B: /[MatMul_538]
Ignore MatMul due to non constant B: /[MatMul_543]


In [137]:
from google.colab import files

In [138]:
files.download("depression-int8.onnx")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
# model = model.to("cpu")

In [ ]:
# !apt-get install git-lfs

In [ ]:
# token="token"

In [ ]:
# model.push_to_hub("xtremedistil-l6-h384-go-emotion", use_auth_token=token)

In [ ]:
# tokenizer.push_to_hub("xtremedistil-l6-h384-go-emotion", use_auth_token=token)